In [47]:
# 1. Imports and topic setup
from newsapi import NewsApiClient
from newsplease import NewsPlease
from sqlalchemy import create_engine, ForeignKey, Column, Integer, String, Text, select
from sqlalchemy.orm import declarative_base, relationship, Session
from articles_orm import Base, Article, ArticleFeatures
from more_itertools import unique_everseen

newsapi = NewsApiClient(api_key='fbfb692eb3844ce59e10eea6069d1161')

topic_list = ['crypto',
              'bitcoin',
              
             'covid',
             'vaccine',
              
             'startups',
              'technology',
              'e-commerce',
              
             'Nokia',
             'Apple',
             'Microsoft']

In [48]:
# 2. Pull 100 initial articles of all chosen topics
# NewsAPI doesn't allow free accounts to access results after number 100.
# NewsAPI also doesn't allow free accounts to access results from before >1month ago.
all_articles = {topic: newsapi.get_everything(q=topic, page_size=100, page=1, language='en') for topic in topic_list}
print(all_articles.keys())

dict_keys(['crypto', 'bitcoin', 'covid', 'vaccine', 'startups', 'technology', 'e-commerce', 'Nokia', 'Apple', 'Microsoft'])


In [49]:
# 3. Check totalResults num and length of downloaded content for integrity
print(*(all_articles[topic]['totalResults'] for topic in topic_list))
print(*(len(all_articles[topic]['articles']) for topic in topic_list))

10783 12612 209799 59465 5532 103724 11026 990 38052 18898
100 100 100 100 100 100 100 100 100 100


In [57]:
# 4. Save urls of every topic to their own dict, but only unique articles (previously some duplicates appeared)
topic_urls = {}
for topic in topic_list:
    cta = all_articles[topic]['articles']
    current_topic_articles = []
    current_topic_articles_titles = []
    for article in cta: 
        if article['title'] not in current_topic_articles_titles: # save only unique
            current_topic_articles += [article]
            current_topic_articles_titles += [article['title']]
    
    current_topic_urls = [article['url'] for article in current_topic_articles]
    topic_urls[topic] = current_topic_urls
print(topic_urls.keys())
print(len(topic_urls['bitcoin']))

dict_keys(['crypto', 'bitcoin', 'covid', 'vaccine', 'startups', 'technology', 'e-commerce', 'Nokia', 'Apple', 'Microsoft'])
94


In [58]:
# 4.1. Test some article
test_url = topic_urls['bitcoin'][0]
article = NewsPlease.from_url(test_url)
print(article.get_dict().keys())
print(article.date_publish)
print(article.maintext)

dict_keys(['authors', 'date_download', 'date_modify', 'date_publish', 'description', 'filename', 'image_url', 'language', 'localpath', 'maintext', 'source_domain', 'text', 'title', 'title_page', 'title_rss', 'url'])
2021-05-12 18:17:08
Just weeks after Tesla started accepting Bitcoin as currency for cars, Elon Musk revealed in a tweet that it will "suspend" the effort. According to the release (Tesla does not appear to have a functioning press office), the company is "concerned about rapidly increasing use of fossil fuels for Bitcoin mining and transactions."
A study last year estimated bitcoin mining used 7.46 GW of electricity on average, and the same study puts the number right now at 16.71 GW. That would represent 147.79 TWh on an annual basis.
Tesla went on to say that it "will not be selling any Bitcoin" (how much it may have already sold of the coins it purchased earlier this year for $1.5 billion is unclear) and said that it will use those holdings for transactions whenever min

In [59]:
# 5. Establish session to database.
engine = create_engine(f"sqlite:///NewsAPI_articles.db")
Base.metadata.create_all(engine)

In [60]:
with Session(engine) as session:
    objects = []
    for topic in topic_list:
        for url in topic_urls[topic]:
            try:
                article = NewsPlease.from_url(url)
            except: #ArticleException: #- some download error happened, skip this article
                continue
            mdl_article = Article(
                parent_topic = topic,
                source = article.source_domain,
                title = article.title,
                description = article.description,
                maintext = article.maintext,
                publication_date = article.date_publish,
                url = url
            )
            print(mdl_article)
            objects += [mdl_article]
        session.bulk_save_objects(objects) # bulk save all topics then flush
        session.commit()
        objects=[]

Article(topic=crypto, title=Frst and Fabric Ventures announce fellowship program for crypto entrepreneurs, description=VC funds Frst and Fabric Ventures are teaming up to create Le Crypto Fellowship. With this program, the two firms want to find the next 10 crypto entrepreneurs in France. And they think they might foster the most promising crypto startups if they don’t have any preconceived idea and team yet. As Pierre Entremont from […] url=http://techcrunch.com/2021/06/03/frst-and-fabric-ventures-announce-fellowship-program-for-crypto-entrepreneurs/)
Article(topic=crypto, title=Crypto exchange Binance faces US money laundering probe, description=Binance, the world's leading cryptocurrency exchange, is reportedly being probed by the DoJ and IRS amid allegations that it is being used to launder money.. url=https://www.engadget.com/crypto-exchange-binance-money-laundering-140037194.html)
Article(topic=crypto, title=Most new NVIDIA RTX gaming GPUs will be crypto-limited, description=NVID

Article(topic=crypto, title=Elon Musk giveth and taketh away, description=Hello friends, and welcome back to Week in Review! Last week, I wrote about Facebook’s never-ending Trump problem. This week, I’m looking at Elon Musk’s wild week of whipping crypto markets. If you’re reading this on the TechCrunch site, you can get this in your inbox from the newsletter page, and follow my tweets @lucasmtny. […] url=http://techcrunch.com/2021/05/16/elon-musk-giveth-and-taketh-away/)
Article(topic=crypto, title=Announcing the Disrupt 2021 Extra Crunch Stage Agenda, description=Disrupt 2020 was an absolute banger of a conference, but we’re not ones to rest on our laurels. We have an incredible Disrupt in store for you this September 21-23, with an amazing lineup of speakers across the hottest of topics, like crypto, health/biotech, SPACs, the creator economy and the usual spread of startup how-to […] url=http://techcrunch.com/2021/06/03/announcing-the-disrupt-2021-extra-crunch-stage-agenda/)
Artic

Article(topic=crypto, title=Opera brings its gaming browser to mobile, description=Opera has begun beta testing its gaming browser on mobile with plans for a public launch in the coming weeks.. url=https://www.engadget.com/opera-gx-mobile-gaming-browser-114556643.html)
Article(topic=crypto, title=A Peter Thiel-backed blockchain platform just invested $10 billion in a new crypto exchange, description=Peter Thiel. AGON Limited Block.one announced on Tuesday that it would launch a crypto exchange called Bullish. It's landed over $10 billi... url=https://markets.businessinsider.com/currencies/news/peter-thiel-blockchain-platform-invested-billions-crypto-exchange-mike-novogratz-2021-5-1030415884)
Article(topic=crypto, title=Ray Dalio says crypto could be a victim of its own success by inviting regulation - and his comments follow tough words from watchdogs, description=Ray Dalio is one of the most successful hedge fund bosses of all time. Thomas Peter/Reuters Ray Dalio said cryptocurrencies

Article(topic=crypto, title=Stocks slip, crypto under pressure amid inflation worries, description=Global stocks slipped and cryptocurrencies sank on Wednesday as a threat of unwanted inflation had investors shy away from assets seen as vulnerable to any removal of monetary stimulus. url=https://www.reuters.com/world/china/global-markets-wrapup-3-2021-05-19/)
Article(topic=crypto, title=Cryptocurrencies bounce back after Sunday plunge, description=Cryptocurrencies rebounded on Monday, reclaiming ground lost during a bout of weekend selling that was fuelled by further signs of a Chinese crackdown on the emerging sector. url=https://www.reuters.com/world/china/crypto-recovery-stalls-miners-eye-exits-2021-05-24/)
Article(topic=crypto, title=Bitcoin edges back from $40,000 as volatility lingers, description=Bitcoin climbed back above $40,000 on Wednesday for the first time this week, before edging off its highs, as recent volatility in the cryptocurrency market showed few signs of dampenin

Article(topic=crypto, title=Bitcoin falls to 3-week low as IRS seeks approval for reporting rules, description=Bitcoin fell to a three-week low on Tuesday amid signs of institutional investor caution, while the head of the U.S. Internal Revenue Service called for lawmakers to provide authority for tighter reporting rules on crypto transfers. url=https://www.reuters.com/business/wealth-manager-ruffer-exited-11-bitcoin-bet-amid-worries-over-risk-2021-06-08/)
Article(topic=crypto, title=South Korea will hit cryptocurrency transactions with a 20% tax starting next year, description=A screen showing prices of bitcoin and other digital currencies at Bithumb in Seoul, South Korea. AP Photo/Ahn Young-joon South Korea will sta... url=https://markets.businessinsider.com/currencies/news/south-korea-tax-cryptocurrency-transactions-20-percent-bitcoin-capital-2021-5-1030476223)
Article(topic=crypto, title=He took $180 million in crypto to the grave. Or did he? A Q&A with the host of 'Exit Scam.', de

...

Article(topic=Microsoft, title=Many of the best Nintendo Switch games are $40 at GameStop right now, description=As part of its Memorial Day sale, GameStop is offering $20 off The Legend of Zelda: Breath of the Wild and Splatoon 2. There are also nice discounts to be had on Dell monitors, iPad Pro keyboards, and the Microsoft Surface Duo. url=https://www.theverge.com/good-deals/2021/5/25/22451937/nintendo-switch-memorial-day-games-sale-dell-monitor-surface-duo-deal)
Article(topic=Microsoft, title=Bill Gates Hoped Epstein Could Help Him Win a Nobel Peace Prize, description=Microsoft co-founder Bill Gates appears to have hopelessly lost control of his efforts to do damage control on his once-budding bromance with fellow billionaire and notorious serial child abuser Jeffrey Epstein. url=https://gizmodo.com/bill-gates-hoped-jeffrey-epstein-could-help-him-schmooz-1846920594)
Article(topic=Microsoft, title=Taking Microsoft’s Surface Laptop 4 for a spin, description=These days, the path of le

Article(topic=Microsoft, title=Now Generally Available: Microsoft's Open Source Java Distribution, 'Microsoft Build of OpenJDK', description="Microsoft has announced general availability of the Microsoft Build of OpenJDK, the open-source version of the Java development kit," reports ZDNet:
The release follows the April preview of the Microsoft Build of OpenJDK, a long-term support distribution of OpenJDK... Microsoft announced general a... url=https://developers.slashdot.org/story/21/05/29/0532204/now-generally-available-microsofts-open-source-java-distribution-microsoft-build-of-openjdk)
Article(topic=Microsoft, title=Pay what you want (UK deal), description=As of May 28, you can pay what you want for the Essential 2021 Excel Wizard Bundle. url=https://mashable.com/uk/shopping/may-28-the-essential-2021-excel-wizard-bundle/)
Article(topic=Microsoft, title=Microsoft is (finally) killing off Internet Explorer entirely, description=Internet Explorer lives on in Edge IE Mode, LTSC Windows 

not a 200 response: 503


Article(topic=Microsoft, title=Microsoft Releases SimuLand, a Lab Environment To Simulate Attacker Tradecraft, description=Microsoft has open-sourced today a tool that can be used to build lab environments where security teams can simulate attacks and verify the detection effectiveness of Microsoft security products. The Record reports: Named SimuLand, the tool was specifically built to help security/IT teams that use ... url=https://news.slashdot.org/story/21/05/20/2332257/microsoft-releases-simuland-a-lab-environment-to-simulate-attacker-tradecraft)
Article(topic=Microsoft, title=Microsoft announces retirement of Internet Explorer, description=Microsoft announces retirement of Internet Explorer url=https://www.rfi.fr/en/business-and-tech/20210521-microsoft-announces-retirement-of-internet-explorer)
Article(topic=Microsoft, title=91% off online class (UK deal), description=The Complete Big Data and Power BI Bundle is on sale for £28.28 as of May 13, saving you 91% on list price. url=h

Article(topic=Microsoft, title=Microsoft Has Turned the Surface Duo Into a Handheld Xbox, description=Microsoft is updating its Xbox Cloud Gaming (xCloud) app for Android, and it includes dual-screen support for the Surface Duo. The Verge reports: The app update allows Surface Duo owners to use a virtual gamepad on one screen of their device and games on the other. It makes the Surface Duo look mor... url=https://games.slashdot.org/story/21/05/24/2015213/microsoft-has-turned-the-surface-duo-into-a-handheld-xbox)
Article(topic=Microsoft, title=Microsoft To Unveil New Version of Windows On June 24, description=After teasing Windows 10's next UI refresh last week, Microsoft confirmed Wednesday that "the next generation of Windows" will be announced on June 24. CNBC reports: Windows, the dominant operating system for personal computers, is the source of 14% of total revenue for Microsoft, one of the most v... url=https://tech.slashdot.org/story/21/06/02/2031234/microsoft-to-unveil-new-vers

In [67]:
# reuse objects list to query all articles and check everything is fine
with Session(engine) as session:
    articles = session.query(Article).all()
print(len(articles))
print(articles[-1].id)

963
963


In [92]:
# for every article from db, apply feature extraction methods (bag of words -> pca/nmf -> save most important features as ArticleFeatures)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from collections import Counter


additional_remove_chars = "'\"`—“’”"
for article in articles:
    # preprocess
    article_text = article.maintext.lower()
    article_text = article_text.translate(str.maketrans('', '', punctuation+additional_remove_chars)) # fastest strip punctuation
    article_tokens = word_tokenize(article_text)
    article_tokens = [token for token in article_tokens if token not in stopwords.words('english')]
    # bag-of-words from now on
    article_bow = Counter(article_tokens) 
    print(article_bow)
    # 

Counter({'crypto': 7, 'fellowship': 5, 'get': 5, 'frst': 4, 'next': 4, 'entrepreneurs': 4, 'idea': 4, 'company': 4, 'vc': 3, 'fabric': 3, 'ventures': 3, 'le': 3, 'program': 3, 'two': 3, 'firms': 3, 'want': 3, 'france': 3, 'think': 3, 'team': 3, 'youll': 3, 'create': 2, 'dont': 2, 'writes': 2, 'build': 2, 'looking': 2, 'nearly': 2, 'already': 2, 'available': 2, 'around': 2, 'started': 2, '€100000': 2, 'one': 2, 'salary': 2, 'long': 2, 'join': 2, 'first': 2, 'isnt': 2, 'funds': 1, 'teaming': 1, 'find': 1, '10': 1, 'might': 1, 'foster': 1, 'promising': 1, 'startups': 1, 'preconceived': 1, 'yet': 1, 'pierre': 1, 'entremont': 1, 'medium': 1, 'post': 1, 'lot': 1, 'opportunities': 1, 'success': 1, 'actively': 1, 'space': 1, 'developers': 1, 'rich': 1, 'therefore': 1, 'step': 1, 'ambition': 1, 'blockchain': 1, 'development': 1, 'defi': 1, 'projects': 1, 'always': 1, 'open': 1, 'source': 1, 'learning': 1, 'resources': 1, 'free': 1, 'web': 1, 'hard': 1, 'prototype': 1, 'right': 1, 'framework': 1

Counter({'musk': 7, 'says': 7, 'doge': 5, 'dogecoin': 5, 'last': 4, 'week': 4, 'sent': 4, 'tesla': 4, 'market': 4, 'bitcoin': 4, 'team': 3, 'make': 3, 'coin': 3, 'lumin': 3, 'cryptocurrency': 3, 'musks': 3, 'seems': 3, 'tweet': 3, 'would': 3, 'elon': 2, 'tweeted': 2, 'efficient': 2, 'price': 2, 'developer': 2, 'time': 2, 'including': 2, 'tweets': 2, 'months': 2, 'first': 2, 'big': 2, 'maybe': 2, 'perhaps': 2, 'worth': 2, 'small': 2, 'carlson': 2, 'company': 2, 'billion': 2, 'weekend': 2, 'suggested': 2, 'thats': 2, 'spoken': 1, 'developers': 1, 'impact': 1, 'predictable': 1, 'moon': 1, 'latest': 1, 'series': 1, 'declarations': 1, 'viral': 1, 'roller': 1, 'coaster': 1, 'past': 1, 'weeks': 1, 'yet': 1, 'caught': 1, 'doges': 1, 'core': 1, 'development': 1, 'surprise': 1, 'michi': 1, 'coloradobased': 1, 'helps': 1, 'maintain': 1, 'yes': 1, 'spacex': 1, 'billionaire': 1, 'spitballing': 1, 'ideas': 1, 'welcomed': 1, 'input': 1, 'intense': 1, 'spotlight': 1, 'recent': 1, 'string': 1, 'dogeboo

Counter({'users': 7, 'playbook': 6, 'work': 6, 'saas': 5, 'get': 5, 'product': 5, 'apps': 4, 'onboarding': 4, 'decade': 4, 'experience': 4, 'consumerization': 3, 'consumer': 3, 'frictionless': 3, 'promise': 3, 'user': 3, 'free': 3, 'incentivize': 3, 'right': 3, 'might': 3, 'like': 3, 'one': 3, 'new': 3, 'enterprise': 2, 'rise': 2, 'could': 2, 'products': 2, 'taught': 2, 'us': 2, 'effectively': 2, 'front': 2, 'next': 2, 'properly': 2, 'necessary': 2, 'true': 2, 'airtable': 2, 'around': 2, 'start': 2, 'hunt': 2, 'story': 2, 'dozen': 2, 'poor': 2, 'level': 2, 'way': 2, 'takes': 2, 'dollars': 2, 'time': 1, 'joined': 1, 'box': 1, 'late': 1, '2012': 1, 'movement': 1, 'well': 1, 'underway': 1, 'clear': 1, 'lessons': 1, 'tactics': 1, 'viral': 1, 'loops': 1, 'social': 1, 'referrals': 1, 'distilled': 1, 'packaged': 1, 'ported': 1, 'subversive': 1, 'great': 1, 'galvanize': 1, 'loyal': 1, 'base': 1, 'wrest': 1, 'fates': 1, 'multimilliondollar': 1, 'contracts': 1, 'suited': 1, 'salespeople': 1, 'pe

Counter({'solana': 14, 'yakovenko': 9, 'says': 9, 'ethereum': 9, 'blockchain': 6, 'proof': 6, 'transactions': 6, 'new': 5, 'like': 5, 'validators': 5, 'network': 5, 'working': 4, 'big': 4, 'systems': 4, 'right': 4, 'miners': 4, 'cryptocurrency': 4, 'stake': 4, 'people': 4, 'one': 4, 'isnt': 3, 'interesting': 3, 'years': 3, 'qualcomm': 3, 'idea': 3, 'speed': 3, 'consensus': 3, 'blockchains': 3, 'thats': 3, 'takes': 3, 'case': 3, 'reason': 3, 'way': 3, 'faster': 3, 'certainly': 3, 'market': 3, 'even': 3, 'transaction': 3, 'go': 3, 'company': 3, 'decentralized': 3, 'set': 3, 'lot': 3, 'build': 3, 'global': 3, 'better': 3, 'linux': 3, 'yet': 2, 'crypto': 2, 'san': 2, 'francisco': 2, 'cafe': 2, 'two': 2, 'reached': 2, 'called': 2, 'work': 2, 'process': 2, 'bitcoins': 2, 'bitcoin': 2, 'developers': 2, 'excited': 2, 'system': 2, 'wherein': 2, 'take': 2, 'need': 2, 'far': 2, 'opportunity': 2, 'meanwhile': 2, 'yesterday': 2, 'saying': 2, 'mindshare': 2, 'billion': 2, 'cap': 2, 'still': 2, 'bots

Counter({'project': 15, 'energy': 15, 'power': 14, 'mining': 13, 'solar': 12, 'crypto': 10, 'said': 10, 'projects': 8, 'advertisement': 8, 'renewable': 7, 'would': 7, 'atlas': 7, 'montana': 6, 'state': 5, 'company': 5, 'mine': 5, 'array': 5, 'vincent': 5, 'new': 5, 'one': 4, 'industries': 4, 'standard': 4, 'million': 4, 'megawatts': 4, 'operation': 4, 'back': 4, 'utilities': 4, 'also': 4, 'hedges': 4, 'people': 4, 'industry': 4, 'largest': 3, 'cryptocurrency': 3, 'get': 3, 'really': 3, 'big': 3, 'around': 3, '2020': 3, 'country': 3, 'called': 3, 'bitcoin': 3, 'center': 3, 'facility': 3, 'thats': 3, 'put': 3, 'building': 3, 'needs': 3, 'seems': 3, 'local': 3, 'forward': 3, 'renewables': 3, 'owner': 3, 'cryptowatt': 3, 'wants': 2, 'build': 2, 'worried': 2, 'impact': 2, 'could': 2, 'installing': 2, 'huge': 2, 'madison': 2, 'river': 2, 'equity': 2, 'butte': 2, 'reported': 2, 'take': 2, 'homes': 2, 'entire': 2, 'end': 2, 'rank': 2, 'fx': 2, 'solutions': 2, 'lot': 2, 'operations': 2, 'permit

...

Counter({'nokia': 9, 'mah': 6, 'c20': 5, 'c30': 4, 'model': 3, 'battery': 3, 'x50': 3, 'though': 3, 'camera': 3, 'plus': 3, 'batteries': 2, 'capacity': 2, 'phone': 2, 'new': 2, '6000': 2, 'dual': 2, 'looks': 2, 'reader': 2, '5000': 2, 'g10': 1, 'g20': 1, '5050': 1, 'biggest': 1, 'far': 1, 'however': 1, 'may': 1, 'surpass': 1, 'belongs': 1, 'basic': 1, 'android': 1, 'go': 1, 'edition': 1, 'series': 1, 'according': 1, 'leakster': 1, 'baidu': 1, 'posted': 1, 'schematic': 1, 'matches': 1, 'rumored': 1, 'two': 1, 'opposite': 1, 'ends': 1, 'spectrum': 1, '–': 1, 'reportedly': 1, 'flagship': 1, '108mp': 1, 'main': 1, 'zeiss': 1, 'optics': 1, 'fingerprint': 1, 'nice': 1, 'find': 1, 'c': 1, 'family': 1, 'targets': 1, 'sub€100': 1, 'segment': 1, 'drawings': 1, 'showing': 1, 'respectively': 1, 'another': 1, 'upcoming': 1, 'thats': 1, 'quite': 1, 'bit': 1, 'tank': 1, '3000': 1, 'one': 1, 'getting': 1, 'vs': 1, 'single': 1, '5mp': 1, 'cam': 1, 'miss': 1, 'fp': 1, 'considering': 1, 'name': 1, 'large

Counter({'amc': 12, 'stock': 9, 'reese': 7, 'current': 6, 'price': 6, 'said': 5, 'target': 5, 'new': 5, 'week': 5, 'shares': 3, 'equity': 3, 'fundamentals': 3, 'trading': 3, 'million': 3, 'raise': 3, 'wrote': 3, 'factset': 3, 'around': 2, 'session': 2, 'analyst': 2, 'theater': 2, 'best': 2, 'meme': 2, 'capital': 2, 'prices': 2, 'touch': 2, 'neutral': 2, 'since': 2, 'lifted': 2, 'levels': 2, 'much': 2, 'opening': 2, 'bell': 2, 'thursday': 2, 'companys': 2, 'closing': 2, 'reduce': 2, 'debt': 2, 'growth': 2, 'part': 2, 'business': 2, 'box': 2, 'office': 2, 'day': 2, 'well': 2, 'large': 2, 'think': 2, '10': 2, 'market': 2, 'dynamics': 2, 'equivalent': 2, 'ratings': 2, 'stocks': 2, 'corp': 2, 'entertainment': 1, 'holdings': 1, 'inc': 1, 'bounced': 1, 'settling': 1, 'lower': 1, 'another': 1, 'volatile': 1, 'friday': 1, 'wedbush': 1, 'alicia': 1, 'movie': 1, 'operator': 1, 'made': 1, 'status': 1, 'raising': 1, 'remained': 1, 'reiterated': 1, 'rating': 1, 'shes': 1, 'march': 1, '2020': 1, '750

Counter({'c20': 9, 'plus': 7, 'also': 4, 'hmd': 3, 'nokia': 2, '11': 2, 'poster': 2, 'phone': 2, 'likely': 2, 'side': 2, 'expect': 2, 'mah': 2, 'ram': 2, 'global': 1, 'unveiled': 1, 'april': 1, 'today': 1, 'announced': 1, 'introduce': 1, 'june': 1, 'company': 1, 'shared': 1, 'weibo': 1, 'revealing': 1, 'design': 1, 'textured': 1, 'back': 1, 'housing': 1, 'two': 1, 'cameras': 1, 'inside': 1, 'circular': 1, 'island': 1, 'theres': 1, 'fingerprint': 1, 'reader': 1, 'wont': 1, 'one': 1, 'screen': 1, 'either': 1, 'youll': 1, 'rely': 1, 'passwordpatternpin': 1, 'unlock': 1, 'upon': 1, 'closer': 1, 'inspection': 1, 'see': 1, 'top': 1, '35mm': 1, 'headphone': 1, 'jack': 1, 'leftside': 1, 'frame': 1, 'home': 1, 'sim': 1, 'card': 1, 'slot': 1, 'power': 1, 'key': 1, 'volume': 1, 'rocker': 1, 'opposite': 1, 'doesnt': 1, 'show': 1, 'us': 1, 'front': 1, 'waterdrop': 1, 'notch': 1, 'display': 1, 'like': 1, 'nonplus': 1, 'variant': 1, 'could': 1, 'bigger': 1, 'diagonal': 1, 'smartphones': 1, 'rumored':

Counter({'android': 20, '12': 14, 'beta': 14, 'os': 11, 'developers': 10, 'program': 8, 'users': 7, 'oneplus': 6, 'developer': 6, 'apps': 5, 'early': 5, 'preview': 5, 'devices': 4, 'get': 4, 'advanced': 4, 'device': 4, 'nokia': 4, 'community': 4, 'google': 3, '2021': 3, 'like': 3, 'versions': 3, 'taste': 3, 'flagship': 3, 'experience': 3, '9': 3, 'pro': 3, 'test': 3, 'right': 3, 'away': 3, 'ready': 3, 'revealed': 2, 'io': 2, 'redesign': 2, 'things': 2, 'intuitive': 2, 'settings': 2, 'material': 2, 'design': 2, 'call': 2, 'operating': 2, 'manufacturers': 2, 'give': 2, 'vivo': 2, 'upcoming': 2, 'also': 2, 'announced': 2, 'ecosystem': 2, 'phone': 2, 'latest': 2, 'one': 2, 'instructions': 2, 'oppo': 2, '1': 2, 'another': 2, 'oem': 2, 'opportunity': 2, 'via': 2, 'bring': 2, 'find': 2, 'x3': 2, 'enroll': 2, 'share': 2, 'zenfone': 2, '8': 2, 'californiabased': 1, 'giant': 1, 'claims': 1, 'radical': 1, 'ui': 1, 'ever': 1, 'since': 1, '50': 1, 'thats': 1, 'much': 1, 'evident': 1, 'whats': 1, 'o

AttributeError: 'NoneType' object has no attribute 'lower'